# Lab 1 - Getting Started

## Project Overview

In this project, you will use both `polars` and `pyspark` to process the data from the MinneMUDAC 2016 competition Dive into Water Data.  While the MinneMUDAC 2016 site, is no longer live, a copy was obtained using the [Wayback Machine (https://web.archive.org) and has been provided in [the overview notebook](./MinneMUDAC_2016_Overview.ipynb).  You should document your work in a Jupyter notebook, which will be used to submit your solution.

## Lab 1 Tasks

In this lab, you will perform the following tasks

1. Download and unzip the data.
2. Investigating the columns in various property data files.

### Task 1 - Data download and unzip

While the download links on the original site no longer work, you can access the data by clicking on [this link](https://mnscu-my.sharepoint.com/:u:/g/personal/bn8210wy_minnstate_edu/EdUePet8JsdKv5aUt9gvjoMBxQhXrOx73WpQyVNwLVDfkA?e=rR8qrc)

1. Move the zip file unto your repository
2. Unzip and move the files into your data folder.

**Hint.** Take a look the the Colab section of any module 5 lecture for an example.

In [19]:
from glob import glob
import polars as pl
import polars.selectors as cs
import re
from functools import reduce      
from operator import mul  


In [20]:
# Perform the tasks listed above then use `glob` to verify the contents/paths.
(parcel_paths := [p for p in sorted(glob('data/**/*.txt', recursive=True)) if 'parcels' in p])

['data\\MinneMUDAC_raw_files\\2002_metro_tax_parcels.txt',
 'data\\MinneMUDAC_raw_files\\2003_metro_tax_parcels.txt',
 'data\\MinneMUDAC_raw_files\\2004_metro_tax_parcels.txt',
 'data\\MinneMUDAC_raw_files\\2005_metro_tax_parcels.txt',
 'data\\MinneMUDAC_raw_files\\2006_metro_tax_parcels.txt',
 'data\\MinneMUDAC_raw_files\\2007_metro_tax_parcels.txt',
 'data\\MinneMUDAC_raw_files\\2008_metro_tax_parcels.txt',
 'data\\MinneMUDAC_raw_files\\2009_metro_tax_parcels.txt',
 'data\\MinneMUDAC_raw_files\\2010_metro_tax_parcels.txt',
 'data\\MinneMUDAC_raw_files\\2011_metro_tax_parcels.txt',
 'data\\MinneMUDAC_raw_files\\2012_metro_tax_parcels.txt',
 'data\\MinneMUDAC_raw_files\\2013_metro_tax_parcels.txt',
 'data\\MinneMUDAC_raw_files\\2014_metro_tax_parcels.txt',
 'data\\MinneMUDAC_raw_files\\2015_metro_tax_parcels.txt']

#### Questions

1. Notice that we have multiple property files, one per year.  What verb(s) will be used to combine these files?
2. Why is it important to compare the columns of these files?
3. Use the `%%bash` cell magic along with `head path` to inspect the first few lines of one of the files, where `...` is replaced with the specific path to one of the files.  How are the columns separated?

> <font color="orange"> Your answer here </font>

In [21]:
# Use `head path` to inspect the first few lines of one of the files.
# Be sure to replace the ... with one of the paths.
#%%bash

#head ...

## Task 2 - Perform a column exploration on the property files

<img src="./img/column_master_file.png" width="800">

**Hints.**

1. Use `glob` to get a list of all the parcel files.
2. Use a list comprehension contains pairs of value of the form `(year, df)` where `df` a `polars` or `pyspark` data frame for each file. Keep in mind that the files are large so we will need to leverage the lazy natures of `polars` and `pyspark`. You will want to use `scan_csv` in `polars`.  On the other hand, `pyspark` is lazy and will do minimal work on this step.
3. Perform a column exploration by creating an indicator summary table, with on row per column and one column per year/file.  The values in the table should be `1` if the column is present in that year's file and `0` otherwise.  You may need to pivot or reshape your data frame to get it into this format.
4. There is one problematic year.  Which year is it and what is the problem?  We will be skipping this year in future analysis.
5. Sort the summary table by the column names and inspect the results.  In particular, look for similar names with different spellings or capitalizations.
6. Identify the common columns that are present in all years.

In [22]:
# Glob/path processing code here

year_pattern = re.compile (r'(\d{4})_metro_tax_parcels')
getyear =lambda path: (n.group(1) if (n := year_pattern. search(path))else None)

In [23]:
[getyear(path) for path in parcel_paths]

['2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015']

In [24]:
(schemas := {
    path: { col: pl.String for col in pl.scan_csv(path, has_header= True, ignore_errors=True, separator= '|').limit(1).collect().columns}
    for path in parcel_paths
})

{'data\\MinneMUDAC_raw_files\\2002_metro_tax_parcels.txt': {'ACRES_DEED': String,
  'ACRES_POLY': String,
  'AGPRE_ENRD': String,
  'AGPRE_EXPD': String,
  'AG_PRESERV': String,
  'BASEMENT': String,
  'BLDG_NUM': String,
  'BLOCK': String,
  'CITY': String,
  'CITY_USPS': String,
  'COOLING': String,
  'COUNTY_ID': String,
  'DWELL_TYPE': String,
  'EMV_BLDG': String,
  'EMV_LAND': String,
  'EMV_TOTAL': String,
  'FIN_SQ_FT': String,
  'GARAGE': String,
  'GARAGESQFT': String,
  'GREEN_ACRE': String,
  'HEATING': String,
  'HOMESTEAD': String,
  'HOME_STYLE': String,
  'LANDMARK': String,
  'LOT': String,
  'MULTI_USES': String,
  'NUM_UNITS': String,
  'OPEN_SPACE': String,
  'OWNER_MORE': String,
  'OWNER_NAME': String,
  'OWN_ADD_L1': String,
  'OWN_ADD_L2': String,
  'OWN_ADD_L3': String,
  'OWN_NAME': String,
  'PARC_CODE': String,
  'PIN': String,
  'PIN_1': String,
  'PLAT_NAME': String,
  'PREFIXTYPE': String,
  'PREFIX_DIR': String,
  'SALE_DATE': String,
  'SALE_VALUE': Str

In [25]:
(indicator_tables :=[ (
    pl.scan_csv(path, has_header= True,separator= '|', schema= schemas[path]). limit(1)
    .unpivot()
    .rename({'variable': 'column'})
    .drop('value')
    .with_columns(pl.lit(1).alias(getyear(path)))
    .collect()
) for path in parcel_paths])

[shape: (75, 2)
 ┌───────────────┬──────┐
 │ column        ┆ 2002 │
 │ ---           ┆ ---  │
 │ str           ┆ i32  │
 ╞═══════════════╪══════╡
 │ ACRES_DEED    ┆ 1    │
 │ ACRES_POLY    ┆ 1    │
 │ AGPRE_ENRD    ┆ 1    │
 │ AGPRE_EXPD    ┆ 1    │
 │ AG_PRESERV    ┆ 1    │
 │ BASEMENT      ┆ 1    │
 │ BLDG_NUM      ┆ 1    │
 │ BLOCK         ┆ 1    │
 │ CITY          ┆ 1    │
 │ CITY_USPS     ┆ 1    │
 │ COOLING       ┆ 1    │
 │ COUNTY_ID     ┆ 1    │
 │ DWELL_TYPE    ┆ 1    │
 │ EMV_BLDG      ┆ 1    │
 │ EMV_LAND      ┆ 1    │
 │ EMV_TOTAL     ┆ 1    │
 │ FIN_SQ_FT     ┆ 1    │
 │ GARAGE        ┆ 1    │
 │ GARAGESQFT    ┆ 1    │
 │ GREEN_ACRE    ┆ 1    │
 │ HEATING       ┆ 1    │
 │ HOMESTEAD     ┆ 1    │
 │ HOME_STYLE    ┆ 1    │
 │ LANDMARK      ┆ 1    │
 │ LOT           ┆ 1    │
 │ MULTI_USES    ┆ 1    │
 │ NUM_UNITS     ┆ 1    │
 │ OPEN_SPACE    ┆ 1    │
 │ OWNER_MORE    ┆ 1    │
 │ OWNER_NAME    ┆ 1    │
 │ OWN_ADD_L1    ┆ 1    │
 │ OWN_ADD_L2    ┆ 1    │
 │ OWN_ADD_L3    ┆ 1  

In [26]:
join_next = lambda left , right: left.join( right, on='column', how="full").drop("column_right")

In [27]:
join_next(indicator_tables[0], indicator_tables[1])

column,2002,2003
str,i32,i32
"""ACRES_DEED""",1,null
"""ACRES_POLY""",1,null
"""AGPRE_ENRD""",1,null
"""AGPRE_EXPD""",1,null
"""AG_PRESERV""",1,null
"""BASEMENT""",1,null
"""BLDG_NUM""",1,1
"""BLOCK""",1,null
"""CITY""",1,1


In [28]:
(indicator_table := 
     reduce(join_next, indicator_tables)
    .filter(pl.col('column').is_not_null())
    .fill_null(0))

column,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""ACRES_DEED""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""ACRES_POLY""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""AGPRE_ENRD""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""AGPRE_EXPD""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""AG_PRESERV""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""BASEMENT""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""BLDG_NUM""",1,1,1,1,1,1,1,1,1,1,1,1,1,1
"""BLOCK""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""CITY""",1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [29]:
indicator_table.select(cs.integer()).sum()

2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
75,34,70,70,70,70,70,70,70,70,70,70,70,70


In [30]:
pl.Config.set_tbl_rows(200)
pl.Config.set_tbl_cols(25)

polars.config.Config

In [31]:
(
    indicator_table
    .drop('2002')
    .drop('2003')
    .with_columns(
        pl.reduce(mul, cs.integer()).alias('in_all')
    )
    .filter(pl.col('in_all')==1)
    .get_column('column')
    .to_list()
)

['ACRES_DEED',
 'ACRES_POLY',
 'AGPRE_ENRD',
 'AGPRE_EXPD',
 'AG_PRESERV',
 'BASEMENT',
 'BLDG_NUM',
 'BLOCK',
 'CITY',
 'CITY_USPS',
 'COOLING',
 'COUNTY_ID',
 'DWELL_TYPE',
 'EMV_BLDG',
 'EMV_LAND',
 'EMV_TOTAL',
 'FIN_SQ_FT',
 'GARAGE',
 'GARAGESQFT',
 'GREEN_ACRE',
 'HEATING',
 'HOMESTEAD',
 'HOME_STYLE',
 'LANDMARK',
 'LOT',
 'MULTI_USES',
 'NUM_UNITS',
 'OPEN_SPACE',
 'OWNER_MORE',
 'OWNER_NAME',
 'OWN_ADD_L1',
 'OWN_ADD_L2',
 'OWN_ADD_L3',
 'PARC_CODE',
 'PIN',
 'PLAT_NAME',
 'PREFIXTYPE',
 'PREFIX_DIR',
 'SALE_DATE',
 'SALE_VALUE',
 'SCHOOL_DST',
 'SPEC_ASSES',
 'STREETNAME',
 'STREETTYPE',
 'SUFFIX_DIR',
 'Shape_Area',
 'Shape_Leng',
 'TAX_ADD_L1',
 'TAX_ADD_L2',
 'TAX_ADD_L3',
 'TAX_CAPAC',
 'TAX_EXEMPT',
 'TAX_NAME',
 'TOTAL_TAX',
 'UNIT_INFO',
 'USE1_DESC',
 'USE2_DESC',
 'USE3_DESC',
 'USE4_DESC',
 'WSHD_DIST',
 'XUSE1_DESC',
 'XUSE2_DESC',
 'XUSE3_DESC',
 'XUSE4_DESC',
 'YEAR_BUILT',
 'Year',
 'ZIP',
 'ZIP4',
 'centroid_lat',
 'centroid_long']

In [32]:
(cols_to_keep :=
 {'ACRES_DEED',
 'ACRES_POLY',
 'AGPRE_ENRD',
 'AGPRE_EXPD',
 'AG_PRESERV',
 'BASEMENT',
 'BLDG_NUM',
 'BLOCK',
 'CITY',
 'CITY_USPS',
 'COOLING',
 'COUNTY_ID',
 'DWELL_TYPE',
 'EMV_BLDG',
 'EMV_LAND',
 'EMV_TOTAL',
 'FIN_SQ_FT',
 'GARAGE',
 'GARAGESQFT',
 'GREEN_ACRE',
 'HEATING',
 'HOMESTEAD',
 'HOME_STYLE',
 'LANDMARK',
 'LOT',
 'MULTI_USES',
 'NUM_UNITS',
 'OPEN_SPACE',
 'OWNER_MORE',
 'OWNER_NAME',
 'OWN_ADD_L1',
 'OWN_ADD_L2',
 'OWN_ADD_L3',
 'PARC_CODE',
 'PIN',
 'PLAT_NAME',
 'PREFIXTYPE',
 'PREFIX_DIR',
 'SALE_DATE',
 'SALE_VALUE',
 'SCHOOL_DST',
 'SPEC_ASSES',
 'STREETNAME',
 'STREETTYPE',
 'SUFFIX_DIR',
 'Shape_Area',
 'Shape_Leng',
 'TAX_ADD_L1',
 'TAX_ADD_L2',
 'TAX_ADD_L3',
 'TAX_CAPAC',
 'TAX_EXEMPT',
 'TAX_NAME',
 'TOTAL_TAX',
 'UNIT_INFO',
 'USE1_DESC',
 'USE2_DESC',
 'USE3_DESC',
 'USE4_DESC',
 'WSHD_DIST',
 'XUSE1_DESC',
 'XUSE2_DESC',
 'XUSE3_DESC',
 'XUSE4_DESC',
 'YEAR_BUILT',
 'Year',
 'ZIP',
 'ZIP4',
 'centroid_lat',
 'centroid_long'})

{'ACRES_DEED',
 'ACRES_POLY',
 'AGPRE_ENRD',
 'AGPRE_EXPD',
 'AG_PRESERV',
 'BASEMENT',
 'BLDG_NUM',
 'BLOCK',
 'CITY',
 'CITY_USPS',
 'COOLING',
 'COUNTY_ID',
 'DWELL_TYPE',
 'EMV_BLDG',
 'EMV_LAND',
 'EMV_TOTAL',
 'FIN_SQ_FT',
 'GARAGE',
 'GARAGESQFT',
 'GREEN_ACRE',
 'HEATING',
 'HOMESTEAD',
 'HOME_STYLE',
 'LANDMARK',
 'LOT',
 'MULTI_USES',
 'NUM_UNITS',
 'OPEN_SPACE',
 'OWNER_MORE',
 'OWNER_NAME',
 'OWN_ADD_L1',
 'OWN_ADD_L2',
 'OWN_ADD_L3',
 'PARC_CODE',
 'PIN',
 'PLAT_NAME',
 'PREFIXTYPE',
 'PREFIX_DIR',
 'SALE_DATE',
 'SALE_VALUE',
 'SCHOOL_DST',
 'SPEC_ASSES',
 'STREETNAME',
 'STREETTYPE',
 'SUFFIX_DIR',
 'Shape_Area',
 'Shape_Leng',
 'TAX_ADD_L1',
 'TAX_ADD_L2',
 'TAX_ADD_L3',
 'TAX_CAPAC',
 'TAX_EXEMPT',
 'TAX_NAME',
 'TOTAL_TAX',
 'UNIT_INFO',
 'USE1_DESC',
 'USE2_DESC',
 'USE3_DESC',
 'USE4_DESC',
 'WSHD_DIST',
 'XUSE1_DESC',
 'XUSE2_DESC',
 'XUSE3_DESC',
 'XUSE4_DESC',
 'YEAR_BUILT',
 'Year',
 'ZIP',
 'ZIP4',
 'centroid_lat',
 'centroid_long'}

In [33]:
from column_data import cols_to_keep
cols_to_keep

['ACRES_DEED',
 'ACRES_POLY',
 'AGPRE_ENRD',
 'AGPRE_EXPD',
 'AG_PRESERV',
 'BASEMENT',
 'BLDG_NUM',
 'BLOCK',
 'CITY',
 'CITY_USPS',
 'COOLING',
 'COUNTY_ID',
 'DWELL_TYPE',
 'EMV_BLDG',
 'EMV_LAND',
 'EMV_TOTAL',
 'FIN_SQ_FT',
 'GARAGE',
 'GARAGESQFT',
 'GREEN_ACRE',
 'HEATING',
 'HOMESTEAD',
 'HOME_STYLE',
 'LANDMARK',
 'LOT',
 'MULTI_USES',
 'NUM_UNITS',
 'OPEN_SPACE',
 'OWNER_MORE',
 'OWNER_NAME',
 'OWN_ADD_L1',
 'OWN_ADD_L2',
 'OWN_ADD_L3',
 'PARC_CODE',
 'PIN',
 'PLAT_NAME',
 'PREFIXTYPE',
 'PREFIX_DIR',
 'SALE_DATE',
 'SALE_VALUE',
 'SCHOOL_DST',
 'SPEC_ASSES',
 'STREETNAME',
 'STREETTYPE',
 'SUFFIX_DIR',
 'Shape_Area',
 'Shape_Leng',
 'TAX_ADD_L1',
 'TAX_ADD_L2',
 'TAX_ADD_L3',
 'TAX_CAPAC',
 'TAX_EXEMPT',
 'TAX_NAME',
 'TOTAL_TAX',
 'UNIT_INFO',
 'USE1_DESC',
 'USE2_DESC',
 'USE3_DESC',
 'USE4_DESC',
 'WSHD_DIST',
 'XUSE1_DESC',
 'XUSE2_DESC',
 'XUSE3_DESC',
 'XUSE4_DESC',
 'YEAR_BUILT',
 'Year',
 'ZIP',
 'ZIP4',
 'centroid_lat',
 'centroid_long']

In [34]:
(column_schema :={col: pl.String for col in cols_to_keep})

{'ACRES_DEED': String,
 'ACRES_POLY': String,
 'AGPRE_ENRD': String,
 'AGPRE_EXPD': String,
 'AG_PRESERV': String,
 'BASEMENT': String,
 'BLDG_NUM': String,
 'BLOCK': String,
 'CITY': String,
 'CITY_USPS': String,
 'COOLING': String,
 'COUNTY_ID': String,
 'DWELL_TYPE': String,
 'EMV_BLDG': String,
 'EMV_LAND': String,
 'EMV_TOTAL': String,
 'FIN_SQ_FT': String,
 'GARAGE': String,
 'GARAGESQFT': String,
 'GREEN_ACRE': String,
 'HEATING': String,
 'HOMESTEAD': String,
 'HOME_STYLE': String,
 'LANDMARK': String,
 'LOT': String,
 'MULTI_USES': String,
 'NUM_UNITS': String,
 'OPEN_SPACE': String,
 'OWNER_MORE': String,
 'OWNER_NAME': String,
 'OWN_ADD_L1': String,
 'OWN_ADD_L2': String,
 'OWN_ADD_L3': String,
 'PARC_CODE': String,
 'PIN': String,
 'PLAT_NAME': String,
 'PREFIXTYPE': String,
 'PREFIX_DIR': String,
 'SALE_DATE': String,
 'SALE_VALUE': String,
 'SCHOOL_DST': String,
 'SPEC_ASSES': String,
 'STREETNAME': String,
 'STREETTYPE': String,
 'SUFFIX_DIR': String,
 'Shape_Area': Stri

In [35]:
# Your lazy polars/pyspark code here

In [36]:
# Further exploration (add cells as needed.)

#### Your conclusions

<font color="orange">
Your thoughts here
</font>

The analysis showed me that 2002-2003 are notably different from the rest of the data. It makes me think they shouldn't be included so that things stay consistent. I also think it's good that we made the schemas we dud because things will be flexible and easier to work with. 